<a href="https://colab.research.google.com/github/dave502/NLP/blob/main/lesson_10/nlp_hw_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
import numpy as np

import unicodedata
import re
import os
import io
from pathlib import Path

Загрузка данных

In [ ]:
%%capture
!wget http://www.manythings.org/anki/rus-eng.zip
!rm ru_en -r
!mkdir ru_en
!unzip rus-eng.zip -d ru_en
!rm rus-eng.zip

In [ ]:
!ls ru_en

_about.txt  rus.txt


In [ ]:
data_path = Path('/content/ru_en/rus.txt')

Функция предобработки текста

In [ ]:
def preprocess_sentence(w):
  w = w.lower().strip()

  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
  w = re.sub(r"([?.!,])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
  w = re.sub(r"[^a-zA-Zа-яА-Я?.!,']+", " ", w)

  w = w.strip()

  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  w = '<start> ' + w + ' <end>'
  return w

In [ ]:
preprocess_sentence("This is example sentence.")

'<start> this is example sentence . <end>'

Функции создания и токенизации датасета

In [ ]:
def create_dataset(data_path: Path, num_examples: int) -> zip:
  lines = data_path.read_text(encoding='UTF-8').strip().split('\n')

  word_pairs = [[preprocess_sentence(sent) for sent in line.split('\t')[:2]] for line in lines[:num_examples]]

  return zip(*word_pairs)

def load_dataset(data_path: Path, num_examples: int) -> tuple:
  # creating cleaned input, output pairs
  target_lang, input_lang = create_dataset(data_path, num_examples)

  input_tensor, input_lang_tokenizer = tokenize(input_lang)
  target_tensor, target_lang_tokenizer = tokenize(target_lang)

  return input_tensor, target_tensor, input_lang_tokenizer, target_lang_tokenizer

def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

  return tensor, lang_tokenizer

Проверка создания датасета

In [ ]:
en, ru = create_dataset(data_path, None)
print(f'len(en) = {len(en)}, len(ru) = {len(ru)}')
print(en[0])
print(ru[0])

len(en) = 451436, len(ru) = 451436
<start> go . <end>
<start> марш ! <end>


Загрузка датасета из файла (в виде массивов), получение токенайзеров 

In [ ]:
NUM_EXAMPLES = 100000
input_tensor, target_tensor, input_lang_tokenizer, target_lang_tokenizer = load_dataset(data_path, NUM_EXAMPLES)

Длины полученных тензоров

In [ ]:
# Calculate max_length of the target tensors
max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]
max_length_targ, max_length_inp

(11, 15)

Разбиение датасета на обучающий и валидационный (массивы)

In [ ]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

80000 80000 20000 20000


Проверка работы токенайзеров для вхоящих и целевых тензоров

In [ ]:
def show_mapping(tokenizer, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, tokenizer.index_word[t]))

print ("Input Language; index to word mapping")
show_mapping(input_lang_tokenizer, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
show_mapping(target_lang_tokenizer, target_tensor_train[0])

Input Language; index to word mapping
1 ----> <start>
4 ----> я
5956 ----> прочла
3761 ----> статью
3 ----> .
2 ----> <end>

Target Language; index to word mapping
1 ----> <start>
4 ----> i
154 ----> read
12 ----> the
2033 ----> article
3 ----> .
2 ----> <end>


Параметры модели

In [ ]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
STEPS_PER_EPOCH = len(input_tensor_train)//BATCH_SIZE
EMBEDDING_DIM = 300
UNITS = 1024
vocab_inp_size = len(input_lang_tokenizer.word_index)+1
vocab_tar_size = len(target_lang_tokenizer.word_index)+1

Преобразование датасетов из массивов в tf тензор

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

Проверка размеров полученного тензора ([<размер батча>, <длина строки данных>])

In [ ]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 15]), TensorShape([64, 11]))

Класс энкодера. На выходе - состояние слоя GRU.

In [ ]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units, batch_size, with_attention=False):
        super(Encoder, self).__init__()

        self.batch_size = batch_size
        self.rnn_units = rnn_units

        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.rnn_units, return_sequences=with_attention, return_state=True, recurrent_initializer='glorot_uniform')

    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state=hidden)
        return state

    def get_zero_hidden_state(self):
        return tf.zeros((self.batch_size, self.rnn_units))

In [ ]:
encoder = Encoder(vocab_inp_size, EMBEDDING_DIM, UNITS, BATCH_SIZE)

Проверка работы энкодера. На выходе - (<размер батча>, <количество GRU ячеек>)

In [ ]:
# sample input
sample_hidden = encoder.get_zero_hidden_state()
sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder Hidden state shape: (batch size, units) (64, 1024)


Класс декодера. На входе данные и состояние энкодера, на выходе - вероятностное распределение словаря + состояние gru декодера 

In [ ]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units, batch_size):
        super(Decoder, self).__init__()

        self.batch_size = batch_size
        self.rnn_units = rnn_units
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.rnn_units, return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)

    def call(self, x, hidden):
        # enc_output shape == (batch_size, max_length, hidden_size)
        x = self.embedding(x) # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        output, state = self.gru(x, initial_state=hidden)
        output = tf.reshape(output, (-1, output.shape[2])) # output shape == (batch_size * 1, hidden_size)
        x = self.fc(output) # output shape == (batch_size, vocab)

        return x, state

In [ ]:
decoder = Decoder(vocab_tar_size, EMBEDDING_DIM, UNITS, BATCH_SIZE)

Проверка работы декодера

In [ ]:
decoder_sample_x, decoder_sample_h = decoder(tf.random.uniform((BATCH_SIZE, 1)), sample_hidden)
print(f'{decoder_sample_x.shape}')
print(f'{decoder_sample_h.shape}')

(64, 7335)
(64, 1024)


Оптимизатор и лосс-функция для модели

In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

def loss_funcion(real, pred):
    
    loss_ = loss_object(real, pred)

    mask = tf.math.logical_not(tf.math.equal(real, 0))
    mask = tf.cast(mask, dtype=loss_.dtype)

    loss_ *= mask

    return tf.reduce_mean(loss_)

Функция шага обучения энкодера, декодера

In [ ]:
@tf.function
def train_step(input, target, encoder_hidden):
    loss = 0

    with tf.GradientTape() as tape:
        encoder_hidden = encoder(input, encoder_hidden)

        decoder_hidden = encoder_hidden

        decoder_input = tf.expand_dims([target_lang_tokenizer.word_index['<start>']] * BATCH_SIZE, 1)

        # Teacher forcing - feeding the target as the next input
        for t in range(1, target.shape[1]):
            # passing enc_output to the decoder
            predictions, decoder_hidden = decoder(decoder_input, decoder_hidden)

            loss += loss_funcion(target[:, t], predictions)

            # using teacher forcing
            decoder_input = tf.expand_dims(target[:, t], 1)

    batch_loss = (loss / int(target.shape[1]))

    variables = encoder.trainable_variables + decoder.trainable_variables

    gradients = tape.gradient(loss, variables)

    optimizer.apply_gradients(zip(gradients, variables))

    return batch_loss

Обучение энкодера, декодера

In [ ]:
EPOCHS = 50

for epoch in range(EPOCHS):
    
    encoder_hidden = encoder.get_zero_hidden_state()
    total_loss = 0

    for (batch, (input, target)) in enumerate(dataset.take(STEPS_PER_EPOCH)):
      batch_loss = train_step(input, target, encoder_hidden)
      total_loss += batch_loss

      if batch % 100 == 0: print(f'Epoch {epoch+1} Batch {batch} Loss {batch_loss:.4f}')

    print(f'Epoch {epoch+1} Batch {batch} Loss {total_loss / STEPS_PER_EPOCH:.4f}')

Epoch 1 Batch 0 Loss 4.6652
Epoch 1 Batch 100 Loss 2.0469
Epoch 1 Batch 200 Loss 1.8178
Epoch 1 Batch 300 Loss 1.6540
Epoch 1 Batch 400 Loss 1.5260
Epoch 1 Batch 500 Loss 1.4188
Epoch 1 Batch 600 Loss 1.4605
Epoch 1 Batch 700 Loss 1.3172
Epoch 1 Batch 800 Loss 1.2932
Epoch 1 Batch 900 Loss 1.1983
Epoch 1 Batch 1000 Loss 1.1648
Epoch 1 Batch 1100 Loss 1.1259
Epoch 1 Batch 1200 Loss 0.8650
Epoch 1 Batch 1249 Loss 1.4526
Epoch 2 Batch 0 Loss 0.8534
Epoch 2 Batch 100 Loss 0.8713
Epoch 2 Batch 200 Loss 0.8075
Epoch 2 Batch 300 Loss 0.8159
Epoch 2 Batch 400 Loss 0.7123
Epoch 2 Batch 500 Loss 0.6909
Epoch 2 Batch 600 Loss 0.7694
Epoch 2 Batch 700 Loss 0.5854
Epoch 2 Batch 800 Loss 0.5862
Epoch 2 Batch 900 Loss 0.6290
Epoch 2 Batch 1000 Loss 0.4932
Epoch 2 Batch 1100 Loss 0.5804
Epoch 2 Batch 1200 Loss 0.5669
Epoch 2 Batch 1249 Loss 0.7140
Epoch 3 Batch 0 Loss 0.4448
Epoch 3 Batch 100 Loss 0.3864
Epoch 3 Batch 200 Loss 0.4182
Epoch 3 Batch 300 Loss 0.3715
Epoch 3 Batch 400 Loss 0.4261
Epoch 3 

Функция перевода на основании предикта 

In [ ]:
def evaluate(sentence):
  attantion_plot = np.zeros((max_length_targ, max_length_inp))

  sentence = preprocess_sentence(sentence)

  inputs = [input_lang_tokenizer.word_index[i] for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length_inp, padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''

  hidden = [tf.zeros((1, UNITS))]
  enc_hidden = encoder(inputs, hidden)
  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([target_lang_tokenizer.word_index['<start>']], 0)

  for t in range(max_length_targ):
    predictions, dec_hidden = decoder(dec_input, dec_hidden)

    # storing the attention weights to plot later on
    predicted_id = tf.argmax(predictions[0]).numpy()
    result += target_lang_tokenizer.index_word[predicted_id] + ' '

    if target_lang_tokenizer.index_word[predicted_id] == '<end>':
      return result, sentence

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)

  return result, sentence

In [ ]:
def translate(sentence):
  result, sentence = evaluate(sentence)

  print('Input: %s' % (sentence))
  print('Predicted translation: {}'.format(result))

Проверка перевода

In [ ]:
translate('Здесь хорошо.')

Input: <start> здесь хорошо . <end>
Predicted translation: it's good here . <end> 


In [ ]:
translate('Я не смогу поехать.')

Input: <start> я не смогу поехать . <end>
Predicted translation: i can't go . <end> 


In [ ]:
translate('Вы еще дома?')

Input: <start> вы еще дома ? <end>
Predicted translation: are you still home ? <end> 


In [ ]:
translate('когда обед?')

Input: <start> когда обед ? <end>
Predicted translation: when's dinner ? <end> 


In [ ]:
translate('где рядом магазин?')

Input: <start> где рядом магазин ? <end>
Predicted translation: where is the soup ? <end> 


*Переводит не идеально, но есть и хорошие попадания*